In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql.types import *
from pyspark.sql.functions import *

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,None,pyspark,idle,,,✔


SparkSession available as 'spark'.


In [2]:
df=spark.read.json("s3://<bucket>/uglyjson.json")
df.printSchema()
df.count()

root
 |-- btid: string (nullable = true)
 |-- id_key: string (nullable = true)
 |-- id_value: string (nullable = true)
 |-- mtime: long (nullable = true)
 |-- operation: string (nullable = true)
 |-- organization: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- type: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- provided_to: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- region: string (nullable = true)
 |-- vbtid: string (nullable = true)
 |-- vendors_provided: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- type: string (nullable = true)

19190

In [3]:
#Identify the fields to flatten
complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])

In [4]:
return_type = ArrayType(MapType(StringType(), StringType()))
@udf(returnType=return_type)
def extract_udf(column_value):
    result=[]
    for i in range(len(column_value)):
       d={}
       d["id"]=column_value[i][0]
       d["type"]=column_value[i][1]
       result.append(d)
    if len(result)==0:
       d={}
       d["id"]=""
       d["type"]=""
       result.append(d)
    return result

In [10]:
#df=df.limit(30)
col_name='provided_to'
df1=df.withColumn(col_name,explode(extract_udf(col_name)))
expanded = [col(col_name).getItem(k).alias(col_name+'_'+k) for k in [ n.name for n in  complex_fields[col_name].elementType]]
df2=df1.select("*", *expanded).drop(col_name)
df2.printSchema()

root
 |-- btid: string (nullable = true)
 |-- id_key: string (nullable = true)
 |-- id_value: string (nullable = true)
 |-- mtime: long (nullable = true)
 |-- operation: string (nullable = true)
 |-- organization: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- type: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- region: string (nullable = true)
 |-- vbtid: string (nullable = true)
 |-- vendors_provided: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- provided_to_id: string (nullable = true)
 |-- provided_to_type: string (nullable = true)

In [11]:
col_name='vendors_provided'
df3=df2.withColumn(col_name,explode(extract_udf(col_name)))
expanded = [col(col_name).getItem(k).alias(col_name+'_'+k) for k in [ n.name for n in  complex_fields[col_name].elementType]]
df3=df3.select("*", *expanded).drop(col_name)
df3.printSchema()

root
 |-- btid: string (nullable = true)
 |-- id_key: string (nullable = true)
 |-- id_value: string (nullable = true)
 |-- mtime: long (nullable = true)
 |-- operation: string (nullable = true)
 |-- organization: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- type: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- region: string (nullable = true)
 |-- vbtid: string (nullable = true)
 |-- provided_to_id: string (nullable = true)
 |-- provided_to_type: string (nullable = true)
 |-- vendors_provided_id: string (nullable = true)
 |-- vendors_provided_type: string (nullable = true)

In [15]:
col_name='organization'
expanded = [col(col_name+'.'+k).alias(col_name+'_'+k) for k in [ n.name for n in  complex_fields[col_name]]]
df4=df3.select("*", *expanded).drop(col_name)
df4.printSchema()

root
 |-- btid: string (nullable = true)
 |-- id_key: string (nullable = true)
 |-- id_value: string (nullable = true)
 |-- mtime: long (nullable = true)
 |-- operation: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- region: string (nullable = true)
 |-- vbtid: string (nullable = true)
 |-- provided_to_id: string (nullable = true)
 |-- provided_to_type: string (nullable = true)
 |-- vendors_provided_id: string (nullable = true)
 |-- vendors_provided_type: string (nullable = true)
 |-- organization_id: string (nullable = true)
 |-- organization_type: string (nullable = true)

In [14]:
df4.select("organization_id","organization_type","provided_to_id",
           "provided_to_type","vendors_provided_id","vendors_provided_type").show(10,False)

+---------------+-----------------+--------------+----------------+-------------------+---------------------+
|organization_id|organization_type|provided_to_id|provided_to_type|vendors_provided_id|vendors_provided_type|
+---------------+-----------------+--------------+----------------+-------------------+---------------------+
|sUAMimn        |vendor           |              |                |sUAMimn            |vendor               |
|Q77GfSw        |contributor      |              |                |sUAMimn            |vendor               |
|lo             |vendor           |Q77GfSw       |contributor     |lo                 |vendor               |
|7sbtFLa        |vendor           |              |                |7sbtFLa            |vendor               |
|c1RfDosd       |client           |c1RfDosd      |client          |7sbtFLa            |vendor               |
|c0Rm1keP       |client           |c0Rm1keP      |client          |7sbtFLa            |vendor               |
|7sbtFLa  